<a href="https://colab.research.google.com/github/AugustWinderickx/Applied_Multivariate_Statistical_Analysis/blob/main/Copy_of_topological_data_analysis_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Topological Data Analysis

In this tutorial you will learn how to apply common topological data analysis (TDA) tools for explorating high-dimensional data. TDA has many uses within data analysis. We will focus on exploration, which is particularly powerful when combined with interactive visualizations. TDA can also be used for feature extraction (i.e., summarize complex data by their topological features), guide feature selection for machine learning (i.e., determine which features distinguish sub-populations within the data), and model interpretation (i.e., investigate how a model treats different aspects of the input-space).

## Practical matters

To run this notebook, either on Google Colaborate or locally, you will have to install several packages. Run the code cells in this section and make sure to restart the kernel when asked. This may take a couple of minutes...

In [ ]:
!pip install giotto-tda multi-mst returns

### Imports

General imports, don't forget to evaluate this cell :) Also: this may take a while.

If this crashes, comment the lines with `multi_mst`. We will then have to skip that specific exercise towards the end of the notebook.

In [ ]:
import pandas as pd
import numpy as np
from multi_mst.k_mst_descent import KMSTDescent
from multi_mst.noisy_mst import NoisyMST
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
import json
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from returns.pipeline import flow
import random
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree

### The data
We will use two small datasets to play with. One is a point-cloud of a horse, and the other is the dataset used in the lecture.

#### The horse

In [ ]:
horse_data = pd.read_csv("http://aida-lab.be/assets/horse.csv")

# We have to take a subsample of the data, otherwise the networkx plotting crashes...
# Comment this line if you want to export to CSV and load in gephi.
horse_data = horse_data.sample(n=2000, random_state=42)

# We'll also add an `id` column to each row which we'll need later.
horse_data['id'] = range(0, len(horse_data))

In [ ]:
horse_data

In [ ]:
sns.scatterplot(horse_data,x="x",y="y")

In [ ]:
sns.scatterplot(horse_data,x="z",y="y")

So apparently:
* `x` is left-right
* `y` is bottom-top
* `z` is back-front

#### The circle(s)
You will recognise this data from the lecture, but not right away...

In this case, the graph was already created beforehand, but we will first plot some of the features.

In [ ]:
circles_data = pd.read_csv("http://aida-lab.be/assets/circles.csv")

circles_data

Let's have a look at what this data looks like. It's weird, but don't worry: it will make sense later.

In [ ]:
sns.scatterplot(circles_data, x="x", y="y", hue="colour_hex", size="r", legend=False)

# From high-dimensional data to a network
## Preamble
### Different graph formats
Depending on the tool that you want to use for _visualising_ the networks, you will need different formats.

In this notebook, we will always create nodes and links as arrays. For example:

`nodes` is an array of arrays. In case of the horse, containing `x`, `y`, and `z` values:
```
[[-1.06305e-02,  8.44801e-01,  4.67552e-01],
 [-2.40724e-02,  8.10356e-01,  4.72199e-01],
 [-1.80217e-03,  2.64261e-01, -5.05758e-01],
 [-3.20978e-02,  7.61185e-01,  3.15659e-01],
 [2.86350e-03,  7.25523e-01,  1.60297e-01]]
```

`links` is an array of 2D-arrays, containing the _source_ and _target_ node. That node is identified by its index in the `nodes` array, **starting from 0**:
```
[[0,1],[0,2],[1,2],[2,3]]
```

#### CSV
Let's start with the simplest output format: the nodes and links are stored in separate CSV files. You will need these, if you for example want to show your data in [gephi](http://www.gephi.org).

To export your data in 2 CSV files, it's easiest to create a Pandas dataframe for each first. In case of the horse data, we _already_ have it in that format, so we don't need to do anything:
```
horse_df.to_csv('nodes.csv', index=False)

links_df = pd.DataFrame(links, columns=["source","target"])
links_df.to_csv('links.csv', index=False)
```


#### D3-based format
For some other applications, we need a single JSON file containing nodes and links. It should look like this:

```
{ "nodes": [
    {"id": 0, "x":1, "y":2},
    {"id": 1, "x":3, "y":4},
    {"id": 2, "x":5, "y":6},
    {"id": 3, "x":7, "y":8}],
  "links": [
    {"source":0,"target":1},
    {"source":0,"target":2},
    {"source":1,"target":2},
    {"source":2,"target":3}]
}
```

Given that we have the structures that we described above, we can create the nodes as follows:
```
nodes_as_json = my_dataframe.to_dict(orient="records")
```

Similarly, to create the links:
```
links_as_json = [{'source': item[0], 'target': item[1]} for item in links]
```

To combine these in a single datastructure:
```
my_graph = {"nodes": nodes_as_json, "links": links_as_json}
```

And finally save them to a file
```
with open('my_graph.json', 'w') as f:
    json.dump(my_graph, f)
```

### Where are files saved?
You can find your saved file in Google colab if you click on the "folder" icon on the left.

In [ ]:
# From Pandas data frame to graph nodes
horse_nodes = horse_data.to_dict(orient="records")
horse_nodes[:5]

## kNN networks
A simple way to construct a network representation from a point cloud is to add edges between each node's k-nearest neighbours. The idea of this approach is that near neighbours better capture the local manifold of the data than just the distances. On the other hand, KNN networks lose density information, as all points will get the same number of edges in the network.

The code below constructs a KNN network on top of the data's minimum spanning tree and computes the correlation of the distances in the network with the point cloud distances.

In [ ]:
horse_distances = pdist(horse_data)
horse_distance_matrix = squareform(horse_distances)

In [ ]:
def create_knn_links(distance_matrix):
  k = 5
  knn_links = []

  for i in range(len(distance_matrix[0])):
      dist_i = distance_matrix[i]

      # Get the indices of the k nearest neighbors (excluding the point itself)
      nearest_indices = np.argsort(dist_i)[1:k+1]  # Exclude the point itself (index 0)

      for j in nearest_indices:
          knn_links.append([i,j,dist_i[j]])

  return knn_links

In [ ]:
horse_knn_links = create_knn_links(horse_distance_matrix)

### Plotting using `networkx`
To draw the network, we can

1. export to 2 CSV files and upload in [gephi](www.gephi.org).
2. export as JSON file and use a custom tool (we won't do that here).
3. show the graph _within this notebook_, using `networkx`.

Below you can find the code to draw the graph using `networkx`. **CAUTION**, plots generated by `networkx` are generally good if the graphs are small (10-20 nodes). We're working with much larger data, so the result will be far from ideal.

In [ ]:
def create_networkx(nodes, links):
  G = nx.Graph()

  for node in nodes:
    G.add_node(node["id"])

  for link in links:
    G.add_edge(link[0],link[1])

  return G

Now we can plot. The following code block may take some time...

In [ ]:
G = create_networkx(horse_nodes, horse_knn_links)
pos = nx.forceatlas2_layout(G, max_iter=200)  # positions for all nodes
# pos = nx.arf_layout(G)
nx.draw(G, pos, node_color='steelblue', alpha=0.5, edge_color='gray', node_size=20)
plt.title('Horse graph')
plt.show()

### Plotting using a custom tool
NOTE: This is just for your reference; you won't have to do this yourself.

To convert our data into the D3 format and save it as a JSON file:

In [ ]:
links_as_json = [{'source': int(item[0]), 'target': int(item[1])} for item in horse_knn_links]

In [ ]:
horse_graph = {"nodes": horse_nodes, "links": links_as_json}

In [ ]:
with open('horse_graph.json', 'w') as f:
    json.dump(horse_graph, f)

### Plotting using gephi
#### Installing/using gephi

Gephi can be downloaded from www.gephi.org.

To load a network, first load the **nodes** from a CSV file. In "File", click "**Import spreadsheet**". In the last step, choose "Undirected" for "Graph Type" and "New workspace".

To load the **edges**, again "File" and "Import spreadsheet". Here also choose "Undirected" for "Graph Type", but "**Append to existing workspace**" instead of "New workspace".

To draw the network, select the "**Choose a layout**" dropdown box on the left, and choose e.g. ForceAtlas2. Try out different layouts.

You can change the **size and colour of the nodes** (and links) on the top left. Choose "unique" if you want to give all nodes the same colour or size; choose "partition" if you have categories; choose "Ranking" if you want to use a numerical feature.

#### Creating the files for use in gephi
Gephi will need two CSV files.

In [ ]:
horse_data.to_csv('nodes.csv', index=False)

df_links = pd.DataFrame(horse_knn_links, columns=["source","target","distance"])
df_links.to_csv('links.csv', index=False)

## Multi-MST networks
Here we generate small deviations from the original distance matrix, and calculate the MST (minimal spanning tree) on that altered matrix. At the end we add those MSTs together.

### Using the multi_mst package

In [ ]:
from multi_mst.noisy_mst import NoisyMST

In [ ]:
projector = NoisyMST(num_trees=10, noise_fraction=1.0).fit(horse_data)
coo_matrix = projector.graph_.tocoo()
sources = coo_matrix.row
targets = coo_matrix.col

In [ ]:
sources[:5]

In [ ]:
targets[:5]

In [ ]:
links = [[s, t] for s, t in zip(sources, targets)]

In [ ]:
links[:5]

Write the code necessary to
1. convert the `links` array into a dataframe
2. save those links in a CSV file
3. save the nodes in a CSV file

And then visualise your graph in gephi.

In [ ]:
# YOUR CODE HERE

### Coding this ourselves
We can do a similar thing without using the `multi-mst` package.

In [ ]:
distances = []
def alter_distances(distances, max_amount = np.std(distances)/50):
    changer = lambda t: t + random.uniform(0, max_amount)
    return np.array([changer(d) for d in distances])

In [ ]:
def calculate_mst(distances):
    X = csr_matrix(squareform(distances))
    mst = minimum_spanning_tree(X)
    return np.nonzero(mst)

In [ ]:
def add_mst_to_links(links, mst):
    for i in range(0,len(mst[0])):
        # links.append('{"source":' + str(mst[0][i]) + ', "target":' + str(mst[1][i]) + '}')
        links.append([mst[0][i], mst[1][i]])
    return links

In [ ]:
links = []

for i in range(10):
    print(i)
    flow(
        horse_distances,
        lambda d: alter_distances(d, max_amount = np.std(horse_distances)),
        lambda d: calculate_mst(d),
        lambda d: add_mst_to_links(links,d)
    )

Let's see what this looks like in gephi: export as CSVs and load in gephi.

In [ ]:
horse_data.to_csv('nodes.csv', index=False)

links_df = pd.DataFrame(links, columns=["source","target"])
links_df.to_csv('links.csv', index=False)

## Mapper network
The mapper networks are fundamentally different from the ones above (see the lecture). Each node in the resulting graph is a _cluster_ of datapoints, not single datapoints.

In spatial domains, topological data analysis is typically used to analyze a signal on a regular grid. For instance, the colour of pixels on an image, or the density of tissue in an CT scan. In these cases, the pixels or voxels provide a notion of connectedness, i.e., pixels that are adjacent to each other are connected. As a result, persistent features can be computed over the signal's values instead of the distance between samples.

For point clouds, this is more difficult, because they do not have an inherrent notion of connectivity. Suppose we perform a filtration over some function $f(v)$, that returns a single value for each data point in a point cloud. Then, instead of edges being included in the simplicial complex as the distance threshold increases, now data points are added to the simplicial complex as the threshold on $f$ increases. But how are these data-points connected? Without that information, we cannot compute the topological features.

The [Mapper](http://diglib.eg.org/bitstream/handle/10.2312/SPBG.SPBG07.091-100/091-100.pdf?sequence=1&isAllowed=y) algorithm provides a solution to this problem. It is able to analyzing a signal on point-cloud data. The algorithm works by (a) defining overlapping segments over a lens (or filter), i.e. the signal of interest. Then, (b) it clusters the data points within each segment. Mapper does not impose any restrictions on which clustering algorithm can be used. The resulting clustering provides the notion of connectivity, it determines which data points in each segment belong together. The clusters become nodes in the resulting mapper graph. Note, though, that the distance threshold which determines which data points belong to a cluster can vary between the lens's segments! Finally, (c) Edges between these nodes are added based on data point overlap between the nodes. There can be overlap because the segments of the lens have overlap.

![Mapper overview](https://gist.githubusercontent.com/JelmerBot/37568240a38c39bf39f20302ed8a130f/raw/9a79ff6e0a2f2c8b79714e5aa4830d99be63a0aa/mapper.png)

The code below computes and visualizes a mapper network for the horse data set. You can play around with the n_intervals and overlap_frac parameters to change the resultion of segments. You can also change columns of the projection to determine which columns are used as filter. Using more than one column tends to work best for the horse, but in general we do not know in advance which filters produce interesting results! Also check what happens when you do not normalize the input data!

In [ ]:
from gtda.mapper import make_mapper_pipeline
from gtda.mapper.filter import Projection
from gtda.mapper.cover import CubicalCover, OneDimensionalCover
from gtda.mapper import plot_static_mapper_graph
from sklearn.preprocessing import StandardScaler
from gtda.mapper.cluster import FirstSimpleGap, FirstHistogramGap

# Normalize dimensions with z-score (mean 0 std 1).
# The x-dimension is small compared to y and z, so the clustering
# does not separate the front-legs and back-legs. By scaling, we
# emphazise patterns in dimensions with small values.
scaler=StandardScaler()

# Define filter function – can be any scikit-learn transformer
# The `Projection` class is named confusingly. It does not
# project the data, it simply returns the specified columns of
# the data. Mapper can deal with more than 1 filter dimension!
filter_func=Projection(columns=[1])

# Define cover, i.e. the segments over the filters.
# Here you can specify the number of segments and their overlap
# ratio.
cover=OneDimensionalCover(n_intervals=11, overlap_frac=0.2)

# Choose clustering algorithm
clusterer=FirstSimpleGap()

# Initialise pipeline
pipe = make_mapper_pipeline(
    clustering_preprocessing=scaler,
    filter_func=filter_func,
    cover=cover,
    clusterer=clusterer,
    verbose=False,
    n_jobs=-1,
)

# Compute the mapper network and visualize it
fig = plot_static_mapper_graph(
    pipe, horse_data.to_numpy(), color_data=horse_data[['y', 'z', 'x']]
)
fig.show(config={'scrollZoom': True})

Have a look at the `gtda.mapper` documentation at https://giotto-ai.github.io/gtda-docs/0.3.1/modules/mapper.html to see what you can change in the code above.

If you use `cover=OneDimensionalCover(n_intervals=11, overlap_frac=0.2)`, can you interpret the picture that comes out?

It can difficult to interpret a Mapper network because we do not have control over the clustering. Sometimes we want to increase or reduce the clustering threshold to show more or less nodes per segment. In addition, which filters are used (either dimensions of the data or functions based on the point cloud) also has a large effect on the resulting network.

## Using a lens
Let's have a look if we add a lens to our network.

First, create a multi-MST graph for the `circles_data` dataset. We only want to do that using the `x` and `y` columns, so first need to create a different dataset with just those.

In [ ]:
circles_data_xy = circles_data.drop(["r","colour","id","colour_hex"], axis=1)
circles_data_xy

In [ ]:
circles_nodes = circles_data.to_dict(orient="records")

In [ ]:
circles_distances = pdist(circles_data_xy)
circles_distance_matrix = squareform(circles_distances)

In [ ]:
links = []

for i in range(20):
    print(i)
    flow(
        circles_distances,
        lambda d: alter_distances(d, max_amount = np.std(circles_distances)),
        lambda d: calculate_mst(d),
        lambda d: add_mst_to_links(links,d)
    )

In [ ]:
circles_data.to_csv('nodes.csv', index=False)

links_df = pd.DataFrame(links, columns=["source","target"])
links_df.to_csv('links.csv', index=False)

If we load this data in gephi, we get the following picture:

<img src="https://aida-lab.pages.gitlab.kuleuven.be/assets/circles_lens_none.png" alt="circles without lens" height="300" />

### Implementing a lens
A very simple way to create a lens _post-hoc_, is to remove links that should not be there.

For example, the size of the datapoints ranges from 0 to 100. We can remove all links where the difference in node size is larger than 20.

We will go through all links, get the nodes, get their sizes and compare them.

In [ ]:
max_difference = 30
filtered_links = [
    link for link in links
    if abs(circles_nodes[link[0]]['colour'] - circles_nodes[link[1]]['colour']) <= max_difference
]

In [ ]:
print(len(links))
print(len(filtered_links))

In [ ]:
links_df = pd.DataFrame(filtered_links, columns=["source","target"])
links_df.to_csv('filtered_links_colour.csv', index=False)

This is the image that we now get from gephi. You clearly see that there is something else going on that you didn't see in the image at the top of this notebook (i.e. where we just plotted `x` and `y`).

<img src="https://aida-lab.pages.gitlab.kuleuven.be/assets/circles_lens_colour.png" alt="circles with colour lens" height="300" />

# Persistent homology

As explained in the lecture, persistent homology analyzes topological features at multiple scales and determines which features persist across a larger range of these scales. A typical way to compute persistent topological features for point clouds are *Vietoris-Rips complexes*. In 2D, this process can be thought of as spheres covering each data point and introducing an edge when spheres start to overlap as their radius increases. Cliques in the resulting network form *simplices*: edges, triangles, tetrahedrons, and their higher dimensional equivalents. Together, the simplices at a single radius form a *simplicial complex* and the sequence of simplicial complexes over the scale form a *filtration*.

![Vietoris-Rips filtration](https://giotto-ai.github.io/gtda-docs/latest/_images/vietoris_rips_point_cloud.gif)

[Image from the Giotto TDA website](https://giotto-ai.github.io/gtda-docs/latest/notebooks/persistent_homology_graphs.html)

The topological features that we are interested in can be computed from the *simplicial complexes* of a *filtration*. Typically, these topological features correspond to [Betti numbers](https://en.wikipedia.org/wiki/Betti_number) and capture the number of connected components (dimension 0), the number of loops (dimension 1), and number of voids (dimension 2 and higher). Using the filtration, we determine when each feature starts to occur and when it dies. For example, looking at the animation above, you can see at which distance the circle closes, and when it dies because it gets filled in with triangles. The difference between a feature's death and birth is their *persistence*. Generally, we assume that are a few features that are more persistent than the others. These features are interpreted to capture the true underlying shape of the point cloud, while the others are attributed to noise.

Note that 0-dimensional persistence diagrams are closely related to single-linkage hierarchical clustering dendrograms. Both constructs describe at which distances certain data-points belong 'together'. Their difference is in their interpretation of a merge. Suppose that there are two connected components, or clusters, that merge at distance some $d$. The dendrogram interprets this merge to mean that both clusters combine and continue to exist as a single entity. The persistence diagram instead sees the merge as the death of one of the components while the other continues to exist.

## Persistent features of a torus

In this example, we show how to use Giotto-TDA to compute the persistence diagram of a point cloud, in this case a torus.

First, we have to construct the point cloud:

In [ ]:
from gtda.plotting import plot_point_cloud

def make_torus(inner_radius = 1, outer_radius = 2, num_samples = 512):
  s = np.random.rand(num_samples) * np.pi * 2
  t = np.random.rand(num_samples) * np.pi * 2
  return np.column_stack([
    (outer_radius+inner_radius*np.cos(s))*np.cos(t),
    (outer_radius+inner_radius*np.cos(s))*np.sin(t),
    inner_radius * np.sin(s)
  ])

torus = make_torus()
plot_point_cloud(torus)

Now we can use Giotto-TDA to compute the persistent features in the first three dimensions. We use a `WeakAlphaPersistence` filtration to speed up the computation. This filtration computes a Vietoris-Rips complex using only the edges in a Delaunay triangulation, which is efficient to create for low-dimensional datasets.

Giotto-TDA is designed with machine learning in mind. It expects the input the be a training-set of point-clouds. We, however, only use a single point cloud at the moment. So, we have to pass a list as input `[torus]` and then look at the first diagram in the output `diagrams[0]`. Note how we specify which dimensions should be computed and that we set `reduced_homology = False`. This latter argument makes Giotto-TDA return the infinitly persistent 0-dimensional simplex. Otherwise, it would have been silently removed.

In the resulting output, we clearly see the persistent features of a torus: one 0-dimensionional component, two 1-dimensional loops, one 2-dimensional void.



In [ ]:
from gtda.plotting import plot_diagram
from gtda.homology import WeakAlphaPersistence

WA = WeakAlphaPersistence(homology_dimensions=[0, 1, 2],
                          reduced_homology=False,
                          n_jobs=-1)
diagrams = WA.fit_transform([torus])

plot_diagram(diagrams[0])

Another way to view a persistence diagrams is as a barcode, where each bar represents a topological feature and indicates when the feature started to exist and when it died. The code below constructs barcodes for the torus.

In [ ]:
ds = [0, 1, 2]
inf_value = 2
for i, d in enumerate(ds):
  plt.subplot(1, len(ds), i+1)
  components = diagrams[0][diagrams[0][:, 2] == d]
  for i, pair in enumerate(components):
    plt.plot([pair[0], pair[1] if not np.isinf(pair[1]) else inf_value], [i, i], 'k-', linewidth=0.5)
  plt.title(f'Barcode for dimension {d}')
  plt.xlabel('distance')
  plt.ylabel('feature id')
  plt.xlim([0, 2.2])

  plt.plot([inf_value, inf_value], plt.ylim(), 'k:', linewidth=1)
  ticks = np.linspace(0, inf_value, 5)
  labels = [
    f'{t}' if t != inf_value else 'inf'
    for t in ticks
  ]
  plt.gca().set_xticks(ticks)
  plt.gca().set_xticklabels(labels)

plt.subplots_adjust(wspace=0.3)
plt.gcf().set_figwidth(10)
plt.show()

## Persistent features of the horse

Using the example above, compute a the persistence diagram for the horse. How many 0, 1, and 2 dimensional features does the horse have? Can you describe how each feature corresponds the horse's shape?

In [ ]:
plot_point_cloud(horse_data.to_numpy())

In [ ]:
# Compute the horse's persistent features here

As expected there is a single persistent 0-dimensional component. There are no clearly persistent loops, but there is one persistent void.

## Persistent features of the circles dataset
Now do the same thing for the circles data. You will want to use `circles_data_xy` instead of `circles_data` because the latter contains non-numeric data.

In [ ]:
# YOUR CODE HERE